In [2]:
#Imports
from io import BytesIO
import os
import open3d as o3d
import random
import requests
import tarfile
import numpy as np
from scipy.spatial import distance
from scipy.spatial.distance import cdist, pdist, squareform

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
def gromov_wasserstein(pc1: np.ndarray, pc2: np.ndarray) -> float:
    def dist_ecc_fast(ecc, u):
        return (np.mean(ecc <= u))

    out = 0
    # Konvertiere die Punktwolken in NumPy-Arrays
    pc1 = np.asarray(pc1.points)
    pc2 = np.asarray(pc2.points)

    # Reshape input matrices if necessary
    if pc1.ndim == 1:
        pc1 = pc1.reshape(-1, 1)
    if pc2.ndim == 1:
        pc2 = pc2.reshape(-1, 1)

    ecc1 = squareform(pdist(pc1)).mean(0)
    ecc2 = squareform(pdist(pc2)).mean(0)
    unique_ecc = np.unique(np.concatenate((ecc1, ecc2)))
    for i in range(unique_ecc.shape[0] - 1):
        u = unique_ecc[i]
        out += (unique_ecc[i + 1] - unique_ecc[i]) * np.abs(dist_ecc_fast(ecc1, u) - dist_ecc_fast(ecc2, u))

    return (0.5 * out)

In [4]:
def chamfer_distance(pc1: np.ndarray, pc2: np.ndarray) -> float:
    dist = cdist(pc1, pc2)
    ch_dist = (np.min(dist, axis=1).mean() + np.min(dist, axis=0).mean()) / 2
    return ch_dist

In [5]:
def average_ratio(pc1: np.ndarray, pc2: np.ndarray, Dist_list: list) -> float:
    d = cdist(pc1, pc2)
    d0 = d.min(0)
    d1 = d.min(1)

    avr = 0
    for i, dist in enumerate(Dist_list):
        avr += (i + 1) * ((d1 <= dist).sum() / pc1.shape[0] + (d0 <= dist).sum() / pc2.shape[0])
    return avr / (len(Dist_list) ** 2 + len(Dist_list))

In [6]:
def ratio(pc1: np.ndarray, pc2: np.ndarray, thr_list=np.array([0.1, 0.5, 1, 2, 4])) -> list:
    d = np.min(cdist(pc1, pc2), axis=1)
    r = (d <= thr_list[:, None]).mean(1)
    return thr_list, r

In [7]:
def create_pcd_from_mesh(mesh):
    mesh.compute_vertex_normals()
    o3d.visualization.draw_geometries([mesh])
    # distribute dots evenly on the surface
    return mesh.sample_points_uniformly(500)

In [8]:
def load_model(link, path):
    # http://ycb-benchmarks.s3-website-us-east-1.amazonaws.com/
    response = requests.get(link)
    tgz_data = BytesIO(response.content)
    # set the current working directory to the script's directory
    script_directory = os.path.dirname(os.path.abspath(__file__))
    os.chdir(script_directory)
    with tarfile.open(fileobj=tgz_data, mode="r:gz") as tar_ref:
        tar_ref.extractall(script_directory)
    # join paths
    model_path = os.path.join(script_directory, path, "clouds", "merged_cloud.ply")
    # load pointcloud
    pcd = o3d.io.read_point_cloud(model_path)
    return pcd

In [9]:
def load_cad_model(model):
    # load model generated in freecad
    return o3d.io.read_point_cloud(model)

In [10]:
def visualize_model(model):
    o3d.visualization.draw_geometries([model])

In [11]:
def get_num_points(model):
    print(len(model.points))

In [12]:
def create_pointcloud_from_coordinates(coordinates):
    # create point cloud with coordinates
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(coordinates)
    return pcd

In [13]:
def get_coordinates(model):
    coordinates = [list(point) for point in model.points]
    # print(coordinates[:50])
    return coordinates

In [14]:
def random_downsampling(model, endpoints):
    # get coordinates of the models
    coordinates = get_coordinates(model)
    # select random points for downsampling
    for i in range(len(coordinates) - endpoints):
        rannumb = random.randint(0, len(coordinates) - 1)
        del coordinates[rannumb]
    point_cloud = create_pointcloud_from_coordinates(coordinates)
    return point_cloud

In [15]:
def farthest_point_sampling1(model, num_points_keep):
    coordinates = np.array(get_coordinates(model))
    retVal = []
    # to make runs comparable
    random.seed(13)
    # generate "random" int
    randint = random.randint(0, len(coordinates) - 1)
    # select random point from model
    retVal = np.append(retVal, coordinates[randint])
    # delete chosen point from original model after it was added to the downsampled cloud
    coordinates = np.delete(coordinates, randint, axis=0)
    while len(retVal) < num_points_keep:
        retValnp = np.array(retVal)
        # berechne die Entfernungen der ausgewählten Punkte zu den ausgewählten Punkten
        eucl_distances = distance.cdist(retValnp, coordinates, 'euclidean')
        # finde den weitesten Punkt heraus
        # füge den am weitesten entfernten Punkt der Liste hinzu
        retVal = np.append(retValnp, np.max(eucl_distances, axis=0))
        # punkt aus der Liste mit den ursprünglichen Koordinaten löschen
        min_distance_index = np.argmax(eucl_distances)
        coordinates = np.delete(coordinates, min_distance_index, axis=0)
    return retVal

In [16]:
def farthest_point_sampling(model, num_points_keep):
    coordinates = np.array(get_coordinates(model))
    retVal = []
    # to make runs comparable
    random.seed(13)
    # generate "random" int
    randint = random.randint(0, len(coordinates) - 1)
    # select random point from model
    retVal.append(coordinates[randint])
    # delete chosen point from original model after it was added to the downsampled cloud
    coordinates = np.delete(coordinates, randint, axis=0)
    while len(retVal) < num_points_keep:
        # Berechne die euklidischen Distanzen der ausgewählten Punkte zu den verbleibenden Punkten
        eucl_distances = distance.cdist(retVal, coordinates, 'euclidean')
        # Finden Sie den Punkt mit der größten Mindestdistanz
        min_mindist = np.min(eucl_distances, axis=0)
        # Finden Sie den Index des Punktes mit der größten Mindestdistanz
        max_min_distance_index = np.argmax(min_mindist)
        # Fügen Sie den am weitesten entfernten Punkt der Liste hinzu
        retVal.append(coordinates[max_min_distance_index])
        # Entfernen Sie den ausgewählten Punkt aus den verbleibenden Koordinaten
        coordinates = np.delete(coordinates, max_min_distance_index, axis=0)
    return create_pointcloud_from_coordinates(np.array(retVal))

In [17]:
# built in function von open3d?
def radius_outlier_removal_call(model):
    return model.remove_radius_outlier(nb_points=5, radius=0.05)

In [38]:
# add noise to pointcloud
def add_noise(model, noisiness):
    points = np.asarray(model.points)
    noise = np.random.normal(0, noisiness, size=points.shape)
    noisy_points = points + noise

    noisy_pc = o3d.geometry.PointCloud()
    noisy_pc.points = o3d.utility.Vector3dVector(noisy_points)
    return noisy_pc

In [18]:
# with median -> aggregation method as parameter??
def create_voxel_grid(model, voxel_size):
    model_points = np.array(get_coordinates(model))
    min_bound = np.min(model_points, axis=0)
    max_bound = np.max(model_points, axis=0)

    dimensions = np.ceil((max_bound - min_bound) / voxel_size).astype(int)

    voxelgrid = np.zeros(dimensions)

    for point in model_points:
        voxel_coordinates = ((point - min_bound) / voxel_size).astype(int)
        # -1 needed in order to avoid index out of bounds
        voxelgrid[tuple(voxel_coordinates - 1)] += 1
    # visualize voxel grid to check if its correct
    # convert voxelgrid to open3d Voxelgrid
    o3d_voxelgrid = o3d.geometry.VoxelGrid.create_from_point_cloud(input=model, voxel_size=voxel_size)
    #o3d.visualization.draw_geometries([o3d_voxelgrid])
    return o3d_voxelgrid


def voxel_filter(model, voxelgrid, voxel_size):
    # list where downsampled points will be saved
    downsampled_points = []
    # iterate over all voxel in the voxelgrid
    for voxel in voxelgrid.get_voxels():
        # get bounds of the voxel
        downsampled_points.extend(is_point_in_voxel(model, voxelgrid, voxel, voxel_size))
    downsampled_points = np.asarray(downsampled_points)
    return o3d.utility.Vector3dVector(downsampled_points)


def aggregate_points(points):
    # Aggregate the points by averaging, taking into account the z coordinate
    if len(points) == 0:
        return points
    aggregated_points = []
    aggregated_points.append(np.mean(points, axis=0))
    return aggregated_points


def is_point_in_voxel(model, voxelgrid, voxel, voxel_size):
    # get center point and see whether a point lies within the given distance/2 of the voxel size from the center
    voxel_center = voxelgrid.get_voxel_center_coordinate(voxel.grid_index)
    points_in_voxel = []
    half_size = voxel_size / 2.0
    # Überprüfe, welche Punkte innerhalb des Voxels liegen
    for point in model.points:
        if np.all(np.abs(point - voxel_center) <= half_size):
            points_in_voxel.append(point)
    points_in_voxel = aggregate_points(points_in_voxel)
    # print(points_in_voxel)
    return points_in_voxel

def create_points_from_voxel(voxel_model):
    # Vektoren in Numpy Array konvertieren
    vector_array = np.asarray(voxel_model)
    
    # Open3D Punktewolke erstellen
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(vector_array)
    
    return point_cloud


In [19]:
import inspect

def point_cloud_to_ply(point_cloud, file_name): 
    # safe downsampled point cloud as ply data
    o3d.io.write_point_cloud(file_name, point_cloud)

In [20]:
import logging
# Logging
logging.basicConfig(filename='downsampling.log', level=logging.INFO, format='%(asctime)s - %(message)s')

# Pfad zum Speichern der Bilder
output_dir = 'point_cloud_images'
os.makedirs(output_dir, exist_ok=True)
#except Exception as e: logging.error(f'Iteration {i+1}: Fehler aufgetreten - {e}')

In [21]:
# to do lege die Bilder mit relaitven Pfad im Repo an!
cone = load_cad_model(r"cone.ply")
sphere = load_cad_model(r"sphere.ply")
cube = load_cad_model(r"cube.ply")
complex_cube = load_cad_model(r"complexCube.ply")
complex_cone = load_cad_model(r"hollowCone.ply")
complex_sphere = load_cad_model(r"complexSphere.ply")
pencil = load_cad_model(r"pencil_fein.ply")
# source: https://sketchfab.com/3d-models/davis-teapot-materialcleanup-547971eaf21d43f2b6cfcb6be0e7bf11
teapot = load_cad_model(r"teapot.ply")
# source: https://sketchfab.com/3d-models/book-ba04f5ac66194341bc7d437fb6c94674
book = load_cad_model(r"book.ply")

In [22]:
# Laufzeittests

In [23]:
from open3d.examples.open3d_example import draw_registration_result
# Auswertung ICP
# Transformiere die Ziel-Punktwolke zur besseren Demonstration
# ich kann hier auch meine Matplotlib Implementierung verwenden!
def icp_algorithm(source, target):
    # Transformierung der Target Punktewolke
    transformation = np.array([[0.86, 0.5, 0.1, 0.5],
                               [-0.5, 0.86, 0.1, 0.5],
                               [0.0, -0.1, 0.99, 0.5],
                               [0.0, 0.0, 0.0, 1.0]])
    target = target.transform(transformation)
    
    # ICP-Algorithmus ausführen
    threshold = 0.01  # Maximale Entfernung für die Zuordnung von Punkten
    initial_transformation = np.identity(4)  # Initiale Schätzung der Transformation
    
    # Open3D ICP Algorithmus
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source, target, threshold, initial_transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPoint())
    
    # Ergebnis grafisch ausgeben lassen
    draw_registration_result(source, target, reg_p2p.transformation)
    
    # evaluierung wie gut der ICP lief
    evaluation = o3d.pipelines.registration.evaluate_registration(
    source, target, threshold, transformation)
    print(evaluation)


In [24]:
# Auswertung der quantitativen Methoden

In [25]:
import time

#Test für Stabilität der Ergebnisse
# Anzahl der Wiederholungen
# Konfiguration des Loggings
log_filename = 'downsampling.log'
#if os.path.exists(log_filename):
  #  os.remove(log_filename)
    
logging.basicConfig(
    filename=log_filename,
    filemode='w',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)


num_iterations = 6
output_dir_pc = 'point_cloud_images'
os.makedirs(output_dir, exist_ok=True)

model_array = [cone, cube,sphere, complex_cone, complex_sphere, pencil, teapot, book]
for model in model_array:
    for i in range(num_iterations):
        try:
                #random downsampling
                start = time.time()
                rd = random_downsampling(model, 75)
                end = time.time()
                elapsed_time = end - start
                logging.info(f'Random Downsampling - Iteration {i+1}: Downsampling durchgeführt, verbleibende Punkte: {len(rd.points)},      Rechenzeit: {elapsed_time:.4f} Sekunden')
                # Bildspeicherung! -> nachschauen was durch das Reshapen hier passiert!
                point_cloud_to_ply(rd, "random downsampling"+str(i+1))                
                # voxelgrid
                start = time.time()
                vx_grid = create_voxel_grid(model, 0.2)

                vx = voxel_filter(model, vx_grid, 0.2)
                end = time.time()
                elapsed_time = end - start
                logging.info(f'Voxel Downsampling - Iteration {i+1}: Downsampling durchgeführt, verbleibende Punkte: {len(vx)},      Rechenzeit: {elapsed_time:.4f} Sekunden')
                    # Bildspeicherung! -> nachschauen was durch das Reshapen hier passiert!
                point_cloud_to_ply(vx, "voxel filter:"+str(i+1))
                
                start = time.time()
                fp= farthest_point_sampling(model,75)
                end = time.time()
                elapsed_time = end - start
                logging.info(f'Farthest Point Downsampling - Iteration {i+1}: Downsampling durchgeführt, verbleibende Punkte: {len(fp.points)}, Rechenzeit: {elapsed_time:.4f} Sekunden')
                    # Bildspeicherung! -> nachschauen was durch das Reshapen hier passiert!
                point_cloud_to_ply(fp, "farthest point downsampling"+str(i+1))
        except Exception as e:
            logging.error(f'Iteration {i+1}: Fehler aufgetreten - {e}')

logging.info('Alle Iterationen abgeschlossen.') 

[Open3D WARNING] Write geometry::PointCloud failed: unknown file extension  for file random downsampling1.


KeyboardInterrupt: 

Sphere Test

In [31]:
visualize_model(sphere)
# random downsampling
sphere_rd= random_downsampling(sphere, int(len(sphere.points)/10 * 4))
sphere_rd_pc = o3d.geometry.PointCloud()
sphere_rd_pc.points = o3d.utility.Vector3dVector(np.asarray(sphere_rd.points))
icp_algorithm(sphere, sphere_rd_pc) # stimmt es hier die Punktewolken so zu nehmen?
#RegistrationResult with fitness=3.998821e-01, inlier_rmse=0.000000e+00, and correspondence_set size of 2035
# voxel grid filter
sphere_voxel_grid = create_voxel_grid(sphere, 0.2)
sphere_voxel_pc = o3d.geometry.PointCloud()
sphere_voxel_pc.points = voxel_filter(sphere, sphere_voxel_grid,0.2)
icp_algorithm(sphere_voxel_pc,sphere)
#RegistrationResult with fitness=1.000000e+00, inlier_rmse=0.000000e+00, and correspondence_set size of 5089
# farthest point downsampling
sphere_fp = farthest_point_sampling(sphere,int(len(sphere.points)/10 * 4))
sphere_fp_pc = o3d.geometry.PointCloud()
sphere_fp_pc.points = o3d.utility.Vector3dVector(np.asarray(sphere_fp.points))
icp_algorithm(sphere, sphere_fp_pc)
#RegistrationResult with fitness=3.998821e-01, inlier_rmse=0.000000e+00, and correspondence_set size of 2035

RegistrationResult with fitness=3.998821e-01, inlier_rmse=0.000000e+00, and correspondence_set size of 2035
Access transformation to get result.
RegistrationResult with fitness=1.000000e+00, inlier_rmse=0.000000e+00, and correspondence_set size of 5089
Access transformation to get result.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Das Handle ist ungültig. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Das Handle ist ungültig. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Der angeforderte Transformationsvorgang wird nicht unterstützt. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Das Handle ist ungültig. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Der angeforderte Transformationsvorgang wird nicht unterstützt. 
RegistrationResult with fitness=3.998821e-01, inlier_rmse=0.000000e+00, and correspondence_set size of 2035
Access transformation to get result.


Complex Sphere Test

In [32]:
visualize_model(complex_sphere)
complex_sphere_rd= random_downsampling(complex_sphere, int(len(complex_sphere.points)/10 * 2))
icp_algorithm(complex_sphere,complex_sphere_rd) # nehme ich hier bei beiden die gedownsamplete Punktewolke?
#RegistrationResult with fitness=2.000000e-01, inlier_rmse=0.000000e+00, and correspondence_set size of 244
# voxel grid filter
complex_sphere_voxel_grid = create_voxel_grid(complex_sphere, 0.2)
complex_sphere_voxel_pc = o3d.geometry.PointCloud()
complex_sphere_voxel_pc.points = voxel_filter(complex_sphere, complex_sphere_voxel_grid,0.2)
icp_algorithm(complex_sphere_voxel_pc,complex_sphere)
#RegistrationResult with fitness=1.000000e+00, inlier_rmse=0.000000e+00, and correspondence_set size of 1220
# farthest point downsampling
complex_sphere_fp = farthest_point_sampling(complex_sphere,int(len(complex_sphere.points)/10 * 4))
complex_sphere_fp_pc = o3d.geometry.PointCloud()
complex_sphere_fp_pc.points = o3d.utility.Vector3dVector(np.asarray(complex_sphere_fp.points))
icp_algorithm(complex_sphere, complex_sphere_fp_pc)
#RegistrationResult with fitness=4.000000e-01, inlier_rmse=0.000000e+00, and correspondence_set size of 488

RegistrationResult with fitness=2.000000e-01, inlier_rmse=0.000000e+00, and correspondence_set size of 244
Access transformation to get result.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Das Handle ist ungültig. 
RegistrationResult with fitness=1.000000e+00, inlier_rmse=0.000000e+00, and correspondence_set size of 1220
Access transformation to get result.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Das Handle ist ungültig. 
RegistrationResult with fitness=4.000000e-01, inlier_rmse=0.000000e+00, and correspondence_set size of 488
Access transformation to get result.


Teapot Test

In [33]:
visualize_model(teapot)
# random downsampling
teapot_rd= random_downsampling(teapot, int(len(teapot.points)/10 * 2))
icp_algorithm(teapot, teapot_rd) # nehme ich hier bei beiden die gedownsamplete Punktewolke?
#RegistrationResult with fitness=7.348961e-01, inlier_rmse=2.310666e-04, and correspondence_set size of 9123
# voxel grid filter
teapot_voxel_grid = create_voxel_grid(teapot, 0.2)
teapot_voxel_pc = o3d.geometry.PointCloud()
teapot_voxel_pc.points = voxel_filter(teapot, teapot_voxel_grid,0.2)
icp_algorithm(teapot_voxel_pc,teapot)
#egistrationResult with fitness=6.467433e-01, inlier_rmse=7.301364e-04, and correspondence_set size of 844
# farthest point downsampling
teapot_fp = farthest_point_sampling(teapot,int(len(teapot.points)/10 * 4))
teapot_fp_pc = o3d.geometry.PointCloud()
teapot_fp_pc.points = o3d.utility.Vector3dVector(np.asarray(teapot_fp.points))
icp_algorithm(teapot, teapot_fp_pc)
#RegistrationResult with fitness=1.000000e+00, inlier_rmse=0.000000e+00, and correspondence_set size of 12414

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Das Handle ist ungültig. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Der angeforderte Transformationsvorgang wird nicht unterstützt. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Der angeforderte Transformationsvorgang wird nicht unterstützt. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Der angeforderte Transformationsvorgang wird nicht unterstützt. 
RegistrationResult with fitness=7.348961e-01, inlier_rmse=2.310666e-04, and correspondence_set size of 9123
Access transformation to get result.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Der angeforderte Transformationsvorgang wird nicht unterstützt. 
RegistrationResult with fitness=6.467433e-01, inlier_rmse=7.301364e-04, and correspondence_set size of 844
Access transformation to get result.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Der angeforderte Transformati

Cube

In [34]:
visualize_model(cube)
# random downsampling
cube_rd= random_downsampling(cube, int(len(cube.points)/10 * 4))
icp_algorithm(cube, cube_rd) # nehme ich hier bei beiden die gedownsamplete Punktewolke?
#RegistrationResult with fitness=4.000000e-01, inlier_rmse=0.000000e+00, and correspondence_set size of 166
# voxel grid filter
cube_voxel_grid = create_voxel_grid(cube, 0.2)
cube_voxel_pc = o3d.geometry.PointCloud()
cube_voxel_pc.points = voxel_filter(cube, cube_voxel_grid,0.2)
icp_algorithm(cube_voxel_pc,cube)
#RegistrationResult with fitness=1.000000e+00, inlier_rmse=0.000000e+00, and correspondence_set size of 415
# farthest point downsampling
cube_fp = farthest_point_sampling(cube,int(len(cube.points)/10 * 4))
cube_fp_pc = o3d.geometry.PointCloud()
cube_fp_pc.points = o3d.utility.Vector3dVector(np.asarray(cube_fp.points))
icp_algorithm(cube, cube_fp_pc)
#RegistrationResult with fitness=4.000000e-01, inlier_rmse=0.000000e+00, and correspondence_set size of 166

RegistrationResult with fitness=4.000000e-01, inlier_rmse=0.000000e+00, and correspondence_set size of 166
Access transformation to get result.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Das Handle ist ungültig. 
RegistrationResult with fitness=1.000000e+00, inlier_rmse=0.000000e+00, and correspondence_set size of 415
Access transformation to get result.
RegistrationResult with fitness=4.000000e-01, inlier_rmse=0.000000e+00, and correspondence_set size of 166
Access transformation to get result.


Complex Cube

In [36]:
visualize_model(complex_cube)
# Random Downsampling
complex_cube_rd= random_downsampling(complex_cube, int(len(complex_cube.points)/10 * 2))
icp_algorithm(complex_cube, complex_cube_rd) # nehme ich hier bei beiden die gedownsamplete Punktewolke?
#RegistrationResult with fitness=1.999208e-01, inlier_rmse=0.000000e+00, and correspondence_set size of 1514
# voxel grid filter
complex_cube_voxel_grid = create_voxel_grid(complex_cube, 0.2)
complex_cube_voxel_pc = o3d.geometry.PointCloud()
complex_cube_voxel_pc.points = voxel_filter(complex_cube, complex_cube_voxel_grid,0.2)
icp_algorithm(complex_cube_voxel_pc,complex_cube)
#RegistrationResult with fitness=1.000000e+00, inlier_rmse=0.000000e+00, and correspondence_set size of 7566
# farthest point downsampling
complex_cube_fp = farthest_point_sampling(complex_cube,int(len(complex_cube.points)/10 * 4))
complex_cube_fp_pc = o3d.geometry.PointCloud()
complex_cube_fp_pc.points = o3d.utility.Vector3dVector(np.asarray(complex_cube_fp.points))
icp_algorithm(complex_cube, complex_cube_fp_pc)
#RegistrationResult with fitness=3.999736e-01, inlier_rmse=0.000000e+00, and correspondence_set size of 3029

RegistrationResult with fitness=1.999208e-01, inlier_rmse=0.000000e+00, and correspondence_set size of 1514
Access transformation to get result.
RegistrationResult with fitness=1.000000e+00, inlier_rmse=0.000000e+00, and correspondence_set size of 7566
Access transformation to get result.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Das Handle ist ungültig. 
RegistrationResult with fitness=3.999736e-01, inlier_rmse=0.000000e+00, and correspondence_set size of 3029
Access transformation to get result.


Book

In [37]:
visualize_model(book)
# Random Downsampling
book_rd= random_downsampling(book, int(len(book.points)/10 * 2))
icp_algorithm(book, book_rd) # nehme ich hier bei beiden die gedownsamplete Punktewolke?
#RegistrationResult with fitness=2.000058e-01, inlier_rmse=4.281361e-05, and correspondence_set size of 27746
# voxel grid filter
book_voxel_grid = create_voxel_grid(book, 0.2)
book_voxel_pc = o3d.geometry.PointCloud()
book_voxel_pc.points = voxel_filter(book, book_voxel_grid,0.2)
icp_algorithm(book_voxel_pc,book)
#RegistrationResult with fitness=1.000000e+00, inlier_rmse=0.000000e+00, and correspondence_set size of 1220
# farthest point downsampling
book_fp = farthest_point_sampling(book,int(len(book.points)/10 * 4))
book_fp_pc = o3d.geometry.PointCloud()
book_fp_pc.points = o3d.utility.Vector3dVector(np.asarray(book_fp.points))
icp_algorithm(book, book_fp_pc)

RegistrationResult with fitness=2.000058e-01, inlier_rmse=4.281361e-05, and correspondence_set size of 27746
Access transformation to get result.


KeyboardInterrupt: 

Cone

In [ ]:
visualize_model(cone)
cone_rd= random_downsampling(cone, int(len(cone.points)/10 * 2))
#sphere_rd = point_cloud_to_ply(sphere_rd_pc, "sphere_rd.ply")
icp_algorithm(cone, cone_rd) # nehme ich hier bei beiden die gedownsamplete Punktewolke?

# voxel grid filter
cone_voxel_grid = create_voxel_grid(cone, 0.2)
cone_voxel_pc = o3d.geometry.PointCloud()
cone_voxel_pc.points = voxel_filter(cone, cone_voxel_grid,0.2)
icp_algorithm(cone_voxel_pc,cone)
#RegistrationResult with fitness=1.000000e+00, inlier_rmse=0.000000e+00, and correspondence_set size of 1220
# farthest point downsampling
cone_fp = farthest_point_sampling(cone,int(len(cone.points)/10 * 4))
cone_fp_pc = o3d.geometry.PointCloud()
cone_fp_pc.points = o3d.utility.Vector3dVector(np.asarray(cone_fp.points))
icp_algorithm(cone, cone_fp_pc)

Complex Cone

In [ ]:
visualize_model(complex_cone)
complex_cone_rd= random_downsampling(complex_cone, int(len(complex_cone.points)/10 * 2))
icp_algorithm(complex_cone, complex_cone_rd) # nehme ich hier bei beiden die gedownsamplete Punktewolke?

# voxel grid filter
complex_cone_voxel_grid = create_voxel_grid(complex_cone, 0.2)
complex_cone_voxel_pc = o3d.geometry.PointCloud()
complex_cone_voxel_pc.points = voxel_filter(complex_cone, complex_cone_voxel_grid,0.2)
icp_algorithm(complex_cone_voxel_pc,complex_cone)
#RegistrationResult with fitness=1.000000e+00, inlier_rmse=0.000000e+00, and correspondence_set size of 1220
# farthest point downsampling
complex_cone_fp = farthest_point_sampling(complex_cone,int(len(complex_cone.points)/10 * 4))
complex_cone_fp_pc = o3d.geometry.PointCloud()
complex_cone_fp_pc.points = o3d.utility.Vector3dVector(np.asarray(complex_cone_fp.points))
icp_algorithm(complex_cone, complex_cone_fp_pc)

Pencil

In [ ]:
visualize_model(pencil)
pencil_rd= random_downsampling(pencil, int(len(pencil.points)/10 * 2))
icp_algorithm(pencil, pencil_rd) # nehme ich hier bei beiden die gedownsamplete Punktewolke?
# voxel grid filter
pencil_voxel_grid = create_voxel_grid(pencil, 0.2)
pencil_voxel_pc = o3d.geometry.PointCloud()
pencil_voxel_pc.points = voxel_filter(pencil, pencil_voxel_grid,0.2)
icp_algorithm(pencil_voxel_pc,pencil)
#RegistrationResult with fitness=1.000000e+00, inlier_rmse=0.000000e+00, and correspondence_set size of 1220
# farthest point downsampling
pencil_fp = farthest_point_sampling(pencil,int(len(pencil.points)/10 * 4))
pencil_fp_pc = o3d.geometry.PointCloud()
pencil_fp_pc.points = o3d.utility.Vector3dVector(np.asarray(pencil_fp.points))
icp_algorithm(pencil, pencil_fp_pc)

In [48]:
# create noisy pointclouds
noisy_sphere = add_noise(sphere,1.3)
visualize_model(noisy_sphere)
visualize_model(random_downsampling(noisy_sphere,int(len(noisy_sphere.points)/10 * 4)))

noisy_complex_sphere =add_noise(complex_sphere,1.3)
visualize_model(noisy_complex_sphere)
visualize_model(random_downsampling(noisy_complex_sphere,int(len(noisy_complex_sphere.points)/10 * 4)))


noisy_teapot = add_noise(teapot,0.1)
visualize_model(noisy_teapot)
visualize_model(random_downsampling(noisy_teapot, int(len(teapot.points)/10 * 4)))


# voxel grid filter
noisy_sphere_grid = create_voxel_grid(noisy_sphere, 0.2)
noisy_sphere_voxel_pc = o3d.geometry.PointCloud()
noisy_sphere_voxel_pc.points = voxel_filter(noisy_sphere, noisy_sphere_grid,0.2)

noisy_complex_sphere_grid = create_voxel_grid(noisy_complex_sphere, 0.2)
noisy_sphere_voxel_pc = o3d.geometry.PointCloud()
noisy_sphere_voxel_pc.points = voxel_filter(noisy_complex_sphere, noisy_complex_sphere_grid,0.2)

noisy_teapot_grid = create_voxel_grid(noisy_teapot, 0.2)
noisy_teapot_voxel_pc = o3d.geometry.PointCloud()
noisy_teapot_voxel_pc.points = voxel_filter(noisy_teapot, noisy_teapot_grid,0.2)

# farthest point downsampling
noisy_sphere_fp = farthest_point_sampling(noisy_sphere,int(len(noisy_sphere.points)/10 * 4))
noisy_sphere_fp_pc = o3d.geometry.PointCloud()
noisy_sphere_fp_pc.points = o3d.utility.Vector3dVector(np.asarray(noisy_sphere_fp.points))
visualize_model(noisy_sphere_fp_pc)

noisy_complex_sphere_fp = farthest_point_sampling(noisy_complex_sphere,int(len(noisy_complex_sphere.points)/10 * 4))
noisy_complex_sphere_fp_pc = o3d.geometry.PointCloud()
noisy_complex_sphere_fp_pc.points = o3d.utility.Vector3dVector(np.asarray(noisy_complex_sphere_fp.points))
visualize_model(noisy_complex_sphere_fp_pc)

noisy_teapot_fp = farthest_point_sampling(noisy_teapot,int(len(noisy_teapot.points)/10 * 4))
noisy_teapot_fp_pc = o3d.geometry.PointCloud()
noisy_teapot_fp_pc.points = o3d.utility.Vector3dVector(np.asarray(noisy_teapot_fp.points))
visualize_model(noisy_teapot_fp_pc)